In [1]:
import pandas as pd
import sys
import os
import re
import numpy as np
from datetime import datetime

In [2]:
# Datetime converting
def conv_datetime(date):
    conv =  datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d')
    return conv
def conv_just_date(date):
    conv = datetime.strptime(date,'%Y-%m-%d')
    return conv

In [3]:
# Top 10 fechas con mayor cantidad de post creados -> Then mapping dates and posts

# Relación entre cantidad de respuestas y sus visitas. -> Then mapping answers and views

# Mapping information and returning data

file = open(f"posts.xml","r")

with open("newfile_mapped.txt","w") as nw:

    for line in file:
        line = line.strip()
        date = (str(re.findall("CreationDate=\".*?\"",line)).replace("CreationDate=\"","").replace("\"",""))
        score = (str(re.findall("Score=\".*?\"",line)).replace("Score=\"","").replace("\"",""))
        post = (str(re.findall("PostTypeId=\".*?\"",line)).replace("PostTypeId=\"","").replace("\"",""))
        views = (str(re.findall("ViewCount=\".*?\"",line)).replace("ViewCount=\"","").replace("\"",""))
        answers = (str(re.findall("AnswerCount=\".*?\"",line)).replace("AnswerCount=\"","").replace("\"",""))
        txt=("{} {} {} {} {}".format(post, date, score, views, answers))
        txt=txt.replace("[","").replace("]","").replace("\'","")
        nw.write("{}\n".format(txt))

In [4]:
# Reducer 1
file1 = open(f"newfile_mapped.txt","r")
fechas=[]
for line in file1:
    line = line.strip()
    line = line.split(" ")
    try: 
        date = conv_datetime(line[1])
        date = conv_just_date(date)
        fechas.append(date)
    except:
        pass
df_fechas=pd.DataFrame(columns=["fechas"])
df_fechas["fechas"]=df_fechas["fechas"].append(pd.Series(fechas))

tabla_conteo=df_fechas["fechas"].value_counts().rename_axis("registros").reset_index(name='cantidad')
tabla10=tabla_conteo.head(10)
tabla10.to_csv("posts_by_date.txt",header=None, index=None, sep=' ',mode='a')

/tmp/ipykernel_11032/3378930259.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_fechas["fechas"]=df_fechas["fechas"].append(pd.Series(fechas))


In [5]:
# Reducer 2
# Relación entre cantidad de respuestas y sus visitas. -> Then mapping answers and views
with open("views_answers.txt","w") as VA:
    file1 = open(f"newfile_mapped.txt","r")
    for line in file1:
        line = line.strip()
        line = line.split(" ")
        try:
            if line[3] and line[4] != 0: 
                VA.write("{} {}\n".format(line[4],line[3]))
        except:
            pass

In [6]:
# Mapper - score - time
file = open(f"posts.xml","r")

with open("Score_time_mapped.txt","w") as nw:

    for line in file:
        line = line.strip()
        
        post_id = (str(re.findall(" Id=\".*?\"",line)).replace(" Id=\"","").replace("\"","")) #Individual Id 
        posttype = (str(re.findall("PostTypeId=\".*?\"",line)).replace("PostTypeId=\"","").replace("\"","")) #Type 1-Question 2-Answer
        parent_post = (str(re.findall("ParentId=\".*?\"",line)).replace("ParentId=\"","").replace("\"","")) #Parent Id
        date = (str(re.findall("CreationDate=\".*?\"",line)).replace("CreationDate=\"","").replace("\"",""))
        score = (str(re.findall("Score=\".*?\"",line)).replace("Score=\"","").replace("\"",""))

        txt=("{},{},{},{},{}".format(post_id, posttype, date, score, parent_post))
        txt=txt.replace("[","").replace("]","").replace("\'","")
        nw.write("{}\n".format(txt))

In [7]:
score_time=open(f"Score_time_mapped.txt","r")
questions_id=[]
questions_dates=[]
answers_parentid=[]
answers_date=[]
answers_score=[]

def date_normalization(date):
    return datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%f')

for line in score_time:
    line=line.strip()
    line=line.split(",")
    try:
        if int(line[1])==2:
            pass
            # TEST print(line[4],line[2],line[3]) # ParentId, date and score if post is type 2
            answers_parentid.append(int(line[4]))
            answers_date.append(date_normalization(line[2]))
            answers_score.append(int(line[3]))
        else:
            # TEST print(line[0],line[2]) #Id and Date if post is type 1
            questions_id.append(int(line[0]))
            questions_dates.append(date_normalization(line[2]))
    except:
        pass

df_questions=pd.DataFrame(columns=["Id","Date"],index=None)
df_questions["Id"]=questions_id
df_questions["Date"]=questions_dates

df_answers=pd.DataFrame(columns=["Id","Date_new","Score"],index=None)
df_answers["Id"]=answers_parentid
df_answers["Date_new"]=answers_date
df_answers["Score"]=answers_score

top_100 = df_answers.sort_values('Score',ascending=False).head(100)

inner_joined = pd.merge(df_questions,top_100,on=["Id"],how="inner")

inner_joined.sort_values('Score',ascending=False)

with open("average_time.txt","w") as VA:
    VA.write("{}".format(pd.DataFrame.mean(inner_joined["Date_new"]-inner_joined["Date"])))